# eICU Collaborative Research Database

# Notebook 2: Demographics and severity of illness in a single patient

The aim of this Notebook is to introduce high level admission details relating to a single patient stay, using the following tables:

- `patient`
- `admissiondx`
- `apacheapsvar`
- `apachepatientresult`
- `apachepredvar`
- `diagnosis`

Before starting this Notebook, you will need to copy the eicu demo database file ('eicu_demo.sqlite3') to the `data` directory.

Documentation on the eICU Collaborative Research Database can be found at: http://eicu-crd.mit.edu/.

## 1. Getting set up

In [1]:
# Import libraries
import pandas as pd
import matplotlib.pyplot as plt
import sqlite3
import os

In [2]:
# Plot settings
%matplotlib inline
plt.style.use('ggplot')
fontsize = 20 # size for x and y ticks
plt.rcParams['legend.fontsize'] = fontsize
plt.rcParams.update({'font.size': fontsize})

In [3]:
# Connect to the database
fn = os.path.join('data','eicu_demo.sqlite3')
con = sqlite3.connect(fn)
cur = con.cursor()

## 2. Display a list of tables

In [4]:
query = \
"""
SELECT type, name
FROM sqlite_master 
WHERE type='table'
ORDER BY name;
"""

list_of_tables = pd.read_sql_query(query,con)

In [5]:
list_of_tables

,type,name
0,table,admissiondrug
1,table,admissiondx
2,table,apacheapsvar
3,table,apachepatientresult
4,table,apachepredvar
5,table,careplancareprovider
6,table,careplaneol
7,table,careplangeneral
8,table,careplangoal
9,table,careplaninfectiousdisease


## 3. Selecting a single patient stay 

### 3.1. The `patient` table

The `patient` table includes general information about the patient admissions (for example, demographics, admission and discharge details). See: http://eicu-crd.mit.edu/eicutables/patient/

### Questions

Use your knowledge from the previous notebook and the online documentation (http://eicu-crd.mit.edu/) to answer the following questions:

- Which column in the `patient` table is distinct for each stay in the ICU (similar to `icustay_id` in MIMIC-III)?
- Which column is unique for each patient (similar to `subject_id` in MIMIC-III)?


In [6]:
# select a single ICU stay
patientunitstayid = 165840

In [7]:
# query to load data from the patient table
query = \
"""
SELECT *
FROM patient
WHERE patientunitstayid = {}
""".format(patientunitstayid)

print(query)


SELECT *
FROM patient
WHERE patientunitstayid = 165840



In [8]:
# run the query and assign the output to a variable
unitstay = pd.read_sql_query(query,con)

In [9]:
# display the first few rows of the dataframe
unitstay.head()

,patientunitstayid,patienthealthsystemstayid,gender,age,ethnicity,hospitalid,wardid,apacheadmissiondx,admissionheight,hospitaladmityear,...,unitstaytype,admissionweight,dischargeweight,unitdischargeyear,unitdischargetime24,unitdischargetime,unitdischargeoffset,unitdischargelocation,unitdischargestatus,uniquepid
0,165840,148008,Male,81,Caucasian,61,120,"Arrest, respiratory (without cardiac arrest)",167,2015,...,admit,92.4,92.4,2015,23:48:00,midnight,75,Step-Down Unit (SDU),Alive,002-12363


In [10]:
# display a complete list of columns
unitstay.columns

Index([u'patientunitstayid', u'patienthealthsystemstayid', u'gender', u'age',
       u'ethnicity', u'hospitalid', u'wardid', u'apacheadmissiondx',
       u'admissionheight', u'hospitaladmityear', u'hospitaladmittime24',
       u'hospitaladmittime', u'hospitaladmitoffset', u'hospitaladmitsource',
       u'hospitaldischargeyear', u'hospitaldischargetime24',
       u'hospitaldischargetime', u'hospitaldischargeoffset',
       u'hospitaldischargelocation', u'hospitaldischargestatus', u'unittype',
       u'unitadmityear', u'unitadmittime24', u'unitadmittime',
       u'unitadmitsource', u'unitvisitnumber', u'unitstaytype',
       u'admissionweight', u'dischargeweight', u'unitdischargeyear',
       u'unitdischargetime24', u'unitdischargetime', u'unitdischargeoffset',
       u'unitdischargelocation', u'unitdischargestatus', u'uniquepid'],
      dtype='object')

In [11]:
# select a limited number of columns to view
columns = ['patientunitstayid','gender','age','unitdischargestatus']
unitstay[columns]

,patientunitstayid,gender,age,unitdischargestatus
0,165840,Male,81,Alive


### Questions

- What year was the patient admitted to the ICU? Which year was he or she discharged?
- What was the status of the patient upon discharge from the unit?

### 3.2. The `admissiondx` table

The `admissiondx` table contains the primary diagnosis for admission to the ICU according to the APACHE scoring criteria. For more detail, see: http://eicu-crd.mit.edu/eicutables/admissiondx/

### 3.3. The `apacheapsvar` table

The `apacheapsvar` table contains the variables used to calculate the Acute Physiology Score (APS) III for patients. APS-III is an established method of summarizing patient severity of illness on admission to the ICU. 

The score is part of the Acute Physiology Age Chronic Health Evaluation (APACHE) system of equations for predicting outcomes for ICU patients. See: http://eicu-crd.mit.edu/eicutables/apacheApsVar/

### 3.4. The `apachepatientresult` table

The `admissiondx` table provides predictions made by the APACHE score (versions IV and IVa), including probability of mortality, length of stay, and ventilation days. See: http://eicu-crd.mit.edu/eicutables/apachePatientResult/

### 3.5. The `apachepredvar` table

The `apachepredvar` table provides variables underlying the APACHE predictions. Acute Physiology Age Chronic Health Evaluation (APACHE) consists of a groups of equations used for predicting outcomes in critically ill patients. See: http://eicu-crd.mit.edu/eicutables/apachePredVar/

### 3.6. The `diagnosis` table

The `diagnosis` table contains a list of diagnoses that were documented for each patient in the Active Diagnosis/Treatment sections of the eCareManager medical record. The corresponding International Classification of Diseases (ICD) codes are also available in this table. See: http://eicu-crd.mit.edu/eicutables/diagnosis/